## 7. Advanced Experimentation: Deep Feature Engineering & Extended Hyperparameter Search

Let's push further with:
1. **Advanced Feature Transformations**: Log, polynomial, binning, target encoding
2. **Extended Hyperparameter Grids**: Broader search space
3. **RandomizedSearchCV**: Search more parameters efficiently

In [ ]:
# --- 7.1 Advanced Feature Engineering V2 ---
print("="*70)
print("ADVANCED FEATURE ENGINEERING V2")
print("="*70)

from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

df_v2 = df.copy()

# --- Feature 1: Log transformations (handle skewed distributions) ---
df_v2['log_size'] = np.log1p(df_v2['size'])
df_v2['log_room_count'] = np.log1p(df_v2['room_count'])

# --- Feature 2: Ratios and proportions ---
df_v2['bathroom_ratio'] = df_v2['bathroom_count'] / (df_v2['room_count'] + 1)  # bathroom per room
df_v2['size_per_bathroom'] = df_v2['size'] / (df_v2['bathroom_count'] + 1)
df_v2['room_density'] = df_v2['room_count'] / (df_v2['size'] / 100)  # rooms per 100m²

# --- Feature 3: Binned features (categorical from continuous) ---
df_v2['size_category'] = pd.cut(df_v2['size'], bins=[0, 60, 100, 150, 250, 600], 
                                 labels=['studio', 'small', 'medium', 'large', 'luxury'])
df_v2['size_category'] = df_v2['size_category'].astype(str)

# --- Feature 4: Target encoding for city (mean price per city) ---
city_price_mean = df_v2.groupby('city')['price'].transform('mean')
city_price_std = df_v2.groupby('city')['price'].transform('std')
df_v2['city_price_mean'] = city_price_mean
df_v2['city_price_std'] = city_price_std

# --- Feature 5: Polynomial features (size^2) ---
df_v2['size_squared'] = df_v2['size'] ** 2
df_v2['size_cubed'] = df_v2['size'] ** 3 / 10000  # scaled

print("New features created:")
new_features = ['log_size', 'log_room_count', 'bathroom_ratio', 'size_per_bathroom', 
                'room_density', 'size_category', 'tier_x_size', 'tier_x_rooms',
                'city_price_mean', 'city_price_std', 'size_squared', 'size_cubed']
print(f"  {new_features}")
print(f"\nTotal new features: {len(new_features)}")

In [ ]:
# --- 7.2 Prepare Enhanced Dataset V2 ---

# Define features for V2 model
features_v2 = ['city', 'size', 'room_count', 'bathroom_count', 'avg_room_size',
               'log_size', 'bathroom_ratio', 'size_per_bathroom', 'room_density',
               'size_category' , 
               'city_price_mean', 'size_squared']

X_v2 = df_v2[features_v2]
y_v2 = df_v2['log_price']

# Updated preprocessing for V2
categorical_v2 = ['city', 'size_category']
numeric_v2 = [f for f in features_v2 if f not in categorical_v2]

preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numeric_v2),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_v2)
    ])

X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(
    X_v2, y_v2, test_size=0.2, random_state=42
)

print(f"Features V2: {len(features_v2)} total")
print(f"  Numeric: {len(numeric_v2)}")
print(f"  Categorical: {len(categorical_v2)}")
print(f"Training samples: {len(X_train_v2)}")

In [ ]:
# --- 7.3 Extended Hyperparameter Search with RandomizedSearchCV ---
print("\n" + "="*70)
print("EXTENDED HYPERPARAMETER SEARCH (RandomizedSearchCV)")
print("="*70)

from scipy.stats import randint, uniform

# Store V2 results
v2_results = {}

# --- 7.3.1 GradientBoosting with Extended Grid ---
print("\n--- GradientBoosting Extended Search ---")
gb_extended_params = {
    'model__n_estimators': randint(100, 500),
    'model__learning_rate': uniform(0.01, 0.2),
    'model__max_depth': randint(3, 10),
    'model__min_samples_split': randint(2, 20),
    'model__min_samples_leaf': randint(1, 10),
    'model__subsample': uniform(0.6, 0.4),
    'model__max_features': ['sqrt', 'log2', None]
}

gb_pipeline_v2 = Pipeline([
    ('prep', preprocessor_v2),
    ('model', GradientBoostingRegressor(random_state=42))
])

gb_random = RandomizedSearchCV(
    gb_pipeline_v2, gb_extended_params, n_iter=50, cv=5, 
    scoring='r2', n_jobs=-1, random_state=42, return_train_score=True
)
print("Searching 50 random combinations...")
gb_random.fit(X_train_v2, y_train_v2)

y_pred_gb_v2 = gb_random.predict(X_test_v2)
r2_gb_v2 = r2_score(y_test_v2, y_pred_gb_v2)
mae_gb_v2 = mean_absolute_error(np.expm1(y_test_v2), np.expm1(y_pred_gb_v2))

gb_cv_train = gb_random.cv_results_['mean_train_score'][gb_random.best_index_]
gb_cv_test = gb_random.cv_results_['mean_test_score'][gb_random.best_index_]

print(f"Best Params: {gb_random.best_params_}")
print(f"R² = {r2_gb_v2:.4f} | MAE = {mae_gb_v2:,.0f} TND")
print(f"CV Train: {gb_cv_train:.4f} | CV Test: {gb_cv_test:.4f} | Gap: {gb_cv_train-gb_cv_test:.4f}")

v2_results['GB_Extended_V2'] = {
    'R2_Test': r2_gb_v2, 'MAE': mae_gb_v2, 
    'CV_Train': gb_cv_train, 'CV_Test': gb_cv_test,
    'Variance_Gap': gb_cv_train - gb_cv_test
}
best_estimators['GB_Extended_V2'] = gb_random.best_estimator_

In [ ]:
# --- 7.3.2 RandomForest Extended (with regularization to reduce overfitting) ---
print("\n--- RandomForest Extended Search (Anti-Overfitting Focus) ---")

rf_extended_params = {
    'model__n_estimators': randint(100, 400),
    'model__max_depth': randint(5, 20),  # Lower depth to reduce overfitting
    'model__min_samples_split': randint(5, 30),  # Higher to reduce overfitting
    'model__min_samples_leaf': randint(2, 15),
    'model__max_features': ['sqrt', 'log2', 0.5, 0.7],
    'model__max_samples': uniform(0.6, 0.3)  # Bootstrap sample ratio
}

rf_pipeline_v2 = Pipeline([
    ('prep', preprocessor_v2),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

rf_random = RandomizedSearchCV(
    rf_pipeline_v2, rf_extended_params, n_iter=50, cv=5,
    scoring='r2', n_jobs=-1, random_state=42, return_train_score=True
)
print("Searching 50 random combinations...")
rf_random.fit(X_train_v2, y_train_v2)

y_pred_rf_v2 = rf_random.predict(X_test_v2)
r2_rf_v2 = r2_score(y_test_v2, y_pred_rf_v2)
mae_rf_v2 = mean_absolute_error(np.expm1(y_test_v2), np.expm1(y_pred_rf_v2))

rf_cv_train = rf_random.cv_results_['mean_train_score'][rf_random.best_index_]
rf_cv_test = rf_random.cv_results_['mean_test_score'][rf_random.best_index_]

print(f"Best Params: {rf_random.best_params_}")
print(f"R² = {r2_rf_v2:.4f} | MAE = {mae_rf_v2:,.0f} TND")
print(f"CV Train: {rf_cv_train:.4f} | CV Test: {rf_cv_test:.4f} | Gap: {rf_cv_train-rf_cv_test:.4f}")

v2_results['RF_Extended_V2'] = {
    'R2_Test': r2_rf_v2, 'MAE': mae_rf_v2,
    'CV_Train': rf_cv_train, 'CV_Test': rf_cv_test,
    'Variance_Gap': rf_cv_train - rf_cv_test
}
best_estimators['RF_Extended_V2'] = rf_random.best_estimator_

In [ ]:
# --- 7.3.3 SVR Extended with RBF kernel optimization ---
print("\n--- SVR Extended Search ---")

svr_extended_params = {
    'model__C': uniform(0.1, 50),
    'model__epsilon': uniform(0.01, 0.3),
    'model__gamma': ['scale', 'auto'] + list(uniform(0.001, 0.1).rvs(5)),
    'model__kernel': ['rbf', 'poly']
}

svr_pipeline_v2 = Pipeline([
    ('prep', preprocessor_v2),
    ('model', SVR())
])

svr_random = RandomizedSearchCV(
    svr_pipeline_v2, svr_extended_params, n_iter=40, cv=5,
    scoring='r2', n_jobs=-1, random_state=42, return_train_score=True
)
print("Searching 40 random combinations...")
svr_random.fit(X_train_v2, y_train_v2)

y_pred_svr_v2 = svr_random.predict(X_test_v2)
r2_svr_v2 = r2_score(y_test_v2, y_pred_svr_v2)
mae_svr_v2 = mean_absolute_error(np.expm1(y_test_v2), np.expm1(y_pred_svr_v2))

svr_cv_train = svr_random.cv_results_['mean_train_score'][svr_random.best_index_]
svr_cv_test = svr_random.cv_results_['mean_test_score'][svr_random.best_index_]

print(f"Best Params: {svr_random.best_params_}")
print(f"R² = {r2_svr_v2:.4f} | MAE = {mae_svr_v2:,.0f} TND")
print(f"CV Train: {svr_cv_train:.4f} | CV Test: {svr_cv_test:.4f} | Gap: {svr_cv_train-svr_cv_test:.4f}")

v2_results['SVR_Extended_V2'] = {
    'R2_Test': r2_svr_v2, 'MAE': mae_svr_v2,
    'CV_Train': svr_cv_train, 'CV_Test': svr_cv_test,
    'Variance_Gap': svr_cv_train - svr_cv_test
}
best_estimators['SVR_Extended_V2'] = svr_random.best_estimator_

In [ ]:
# --- 7.5 Final Comparison: V1 (Original) vs V2 (Extended) ---
print("\n" + "="*70)
print("FINAL COMPARISON: Original (V1) vs Extended (V2)")
print("="*70)

# V2 Results DataFrame
v2_comparison = pd.DataFrame(v2_results).T
for col in ['R2_Test', 'MAE']:
    if col in v2_comparison.columns:
        v2_comparison[col] = pd.to_numeric(v2_comparison[col], errors='coerce')
v2_comparison = v2_comparison.sort_values('R2_Test', ascending=False)

print("\n--- V2 (Extended Features + Deep Hyperparameter Search) ---")
print(v2_comparison[['R2_Test', 'MAE', 'Variance_Gap']].round(4).to_string())

# Overall comparison
print("\n" + "="*70)
print("HEAD-TO-HEAD: Best V1 vs Best V2")
print("="*70)

best_v1_r2 = r2_voting  # Original voting ensemble
best_v2_name = v2_comparison['R2_Test'].idxmax()
best_v2_r2 = v2_comparison.loc[best_v2_name, 'R2_Test']
best_v2_mae = v2_comparison.loc[best_v2_name, 'MAE']

print(f"\nOriginal Voting Ensemble (V1):")
print(f"  R² = {best_v1_r2:.4f} | MAE = {mae_voting:,.0f} TND")

print(f"\nBest V2 Model ({best_v2_name}):")
print(f"  R² = {best_v2_r2:.4f} | MAE = {best_v2_mae:,.0f} TND")

improvement = (best_v2_r2 - best_v1_r2) * 100
print(f"\n{'🚀' if improvement > 0 else '⚠️'} R² Change: {improvement:+.2f}%")

if improvement > 0:
    print(f"\n✅ SUCCESS! V2 improved by {improvement:.2f}%")
    print(f"   New features and extended hyperparameter search helped!")
else:
    print(f"\n⚠️ V2 did not outperform V1. Original model remains champion.")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² comparison
ax1 = axes[0]
all_r2 = {'Original_Voting (V1)': best_v1_r2}
for name in v2_comparison.index:
    all_r2[f'{name}'] = v2_comparison.loc[name, 'R2_Test']
    
sorted_models = sorted(all_r2.items(), key=lambda x: x[1])
names, values = zip(*sorted_models)
colors = ['gold' if v == max(values) else ('green' if 'V1' in n else 'steelblue') for n, v in sorted_models]
ax1.barh(names, values, color=colors)
ax1.axvline(x=best_v1_r2, color='red', linestyle='--', alpha=0.7, label=f'V1 Baseline: {best_v1_r2:.4f}')
ax1.set_xlabel('R² Score')
ax1.set_title('V1 vs V2 Model Comparison')
ax1.legend()

# Variance Gap (bias-variance tradeoff)
ax2 = axes[1]
variance_data = {k: v.get('Variance_Gap', np.nan) for k, v in v2_results.items() if 'Variance_Gap' in v}
if variance_data:
    names_var = list(variance_data.keys())
    gaps = list(variance_data.values())
    colors_var = ['red' if g > 0.05 else 'green' for g in gaps]
    ax2.barh(names_var, gaps, color=colors_var)
    ax2.axvline(x=0.05, color='orange', linestyle='--', label='Overfitting Threshold')
    ax2.set_xlabel('Variance Gap (Train R² - Test R²)')
    ax2.set_title('Bias-Variance Analysis (Lower = Better)')
    ax2.legend()

plt.tight_layout()
plt.show()